## 네이버 블로그 크롤링

In [18]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
import pickle
from selenium.webdriver.common.by import By

In [19]:
query_txt = "연금"

In [20]:
file_path = "C:\\Users\\cho03\\Downloads\\세미나\\크롤링 교육\\code\\Crawling\\"
path = file_path + 'chromedriver.exe'
driver = webdriver.Chrome(path)

C:\Users\cho03\AppData\Local\Temp\ipykernel_18920\1463534551.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [21]:
driver.get('https://www.naver.com')
time.sleep(1)

element = driver.find_element(By.ID, 'query')

element.send_keys(query_txt)
element.submit()

driver.find_element(By.LINK_TEXT, "VIEW").click()
time.sleep(1)

driver.find_element(By.LINK_TEXT, "블로그").click()
time.sleep(1)

driver.find_element(By.XPATH, """//*[@id="snb"]/div[1]/div/div[2]/a""").click()
time.sleep(1)
print("검색옵션 버튼")

검색옵션 버튼


In [22]:
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]""").click()
time.sleep(1)
print("기간의 직접입력 버튼")

기간의 직접입력 버튼


In [23]:
# 시작기간(data_from) 설정
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[20]/a""").click()
time.sleep(1)
print("년")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)
print("월")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)
print("일")

년
월
일


In [24]:
# 종료기간(data_to) 설정
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a""").click()
time.sleep(1)

In [25]:
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[20]/a""").click()
time.sleep(1)
print("년")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[12]/a""").click()
time.sleep(1)
print("월")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[31]/a""").click()
time.sleep(1)
print("일")

년
월
일


In [26]:
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button""").click()
time.sleep(1)
print("적용")

적용


In [27]:
import datetime

def doScrollDown(whileSeconds): 
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break

In [28]:
doScrollDown(3)

In [29]:
title_list = []
url_list = []
for i in range(1,11):
    title = driver.find_element(By.XPATH,'//*[@id="sp_blog_{}"]/div/div/a'.format(i)).text
    title2 = driver.find_element(By.XPATH, '//*[@id="sp_blog_{}"]/div/div/a'.format(i))
    url = title2.get_attribute('href')
    
    title_list.append(title)
    url_list.append(url)
    if (i % 10) == 0:
        print(i+1)

11


In [30]:
new_doc = []
for i in range(len(url_list)):
    url_path = url_list[i]
    driver.switch_to.window(driver.window_handles[0])
    driver.execute_script("window.open('{}')".format(url_path))
    driver.switch_to.window(driver.window_handles[1])
    time.sleep(1)
    iframes = driver.find_elements(By.TAG_NAME, 'iframe')
    if len(iframes) > 0:
        driver.switch_to.frame(0)
        html = driver.page_source
        soup = BS(html, 'html.parser')
        try:
            a = soup.find('div', class_= 'se-main-container').get_text()
        except:
            a = soup.find('div', id='postListBody')
            a = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣]", "", str(a))
        new_doc.append(a)
        driver.switch_to.default_content()
    
    else:
        new_doc.append(' ')
    driver.close()
    time.sleep(1)
    print(i)

0
1
2
3
4
5
6
7
8
9


In [31]:
raw_data = pd.DataFrame()
raw_data['title'] = title_list
raw_data['doc'] = new_doc
raw_data['url'] = url_list
raw_data['ch'] = 'naver'
raw_data['ch2'] = 'blog'

In [32]:
f = open(file_path + "naver_blog_.pkl", "wb")
pickle.dump(raw_data,f)

f.close()

In [33]:
f = open(file_path + "naver_blog_.pkl", "rb")
temp_file = pickle.load(f)
f.close()

temp_file

,title,doc,url,ch,ch2
0,미래에셋증권 개인연금저축 계좌개설 가입 후기,,https://namsieon.com/5807,naver,blog
1,개인형 퇴직연금 IRP로 준비하는 연말정산 세액공제,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​안녕하세요. 재테크하는제...,https://blog.naver.com/alswl09100/222960736259,naver,blog
2,개인형 퇴직연금 IRP 효율적으로 노후대비 하기,\n\n\n\n\n올해가 시작된 게 엊그제 같은데 어느새 연말을 준비하고 있는 것 ...,https://blog.naver.com/eatlunche/222943537559,naver,blog
3,국민연금 제도와 종류 : 노후준비를 위한 IRP 퇴직연금 운용법,\n\n\n\n\n국민연금 종류와 노후준비개인형 IRP 퇴직연금 운용방법\n\n\n...,https://blog.naver.com/nara212/222922379273,naver,blog
4,주택연금제도 간단히 알아보기,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​많은 분들이 부동산이나 ...,https://blog.naver.com/piancakorea/222967324504,naver,blog
5,연금저축 세액공제 한도 및 혜택 변경되는 내용 정리,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n연금저축 세액공제 한도 및...,https://blog.naver.com/kimgr1010/222923728854,naver,blog
6,연말정산 세액공제 개인형퇴직연금 IRP 퇴직연금 정리,\n\n\n\n\n연말정산에서 세금을 절약할 수 있는공제 방법은 크게 둘로 나눌 수...,https://blog.naver.com/vipro/222962934879,naver,blog
7,"기초연금 :: 금액, 나이, 수급자격, 신청 방법 (ft. 국민연금...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n​안녕하세요. 긍정스텔라입...,https://blog.naver.com/greenher/222964264257,naver,blog
8,"고령농업인을 위한 농지연금제도 (가입요건, 지급방식등)",\n\n\n\n\n안녕하세요~부동산을 공부하는 쑥떡입니다. 저희부모님은 시골에서 농...,https://blog.naver.com/julian1986/222898928339,naver,blog
9,30년납 개인연금저축 수익률 어떻게 될까?,\n\n\n\n\n​2003년 가입해 20년을 납입하고 앞으로 10년을 더 납입해야...,https://blog.naver.com/ishiseido77/222955995100,naver,blog


## 네이버 카페 크롤링

In [41]:
from selenium import webdriver
from bs4 import BeautifulSoup as BS
import pandas as pd
import time
import re
import pickle
from selenium.webdriver.common.by import By

In [1]:
file_path = "C:\\Users\\cho03\\Downloads\\세미나\\code\\Crawling\\"
path = file_path + 'chromedriver.exe'
driver = webdriver.Chrome(path)

NameError: name 'webdriver' is not defined

In [ ]:
query_txt = "연금"

In [44]:
driver.get('https://www.naver.com')
time.sleep(1)

element = driver.find_element(By.ID, 'query')
element.send_keys(query_txt)
element.submit()

In [45]:
driver.find_element(By.LINK_TEXT, "VIEW").click()
time.sleep(1)

driver.find_element(By.LINK_TEXT, "카페").click()
time.sleep(1)

driver.find_element(By.XPATH, """//*[@id="snb"]/div[1]/div/div[2]/a""").click()
time.sleep(1)
print("검색옵션 버튼")

driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[1]/a[9]""").click()
time.sleep(1)
print("기간의 직접입력 버튼")

검색옵션 버튼
기간의 직접입력 버튼


In [46]:
# 시작기간(data_from) 설정
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[33]/a""").click()
time.sleep(1)
print("년")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)
print("월")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)
print("일")

년
월
일


In [47]:
# 종료기간 누르기
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div[1]/span[3]/a""").click()
time.sleep(1)

In [48]:
# 종료기간(data_to) 설정
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[33]/a""").click()
time.sleep(1)
print("년")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[1]/a""").click()
time.sleep(1)
print("월")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[31]/a""").click()
time.sleep(1)
print("일")
driver.find_element(By.XPATH, """//*[@id="snb"]/div[2]/ul/li[4]/div/div[2]/div[3]/button""").click()

년
월
일


In [50]:
import datetime

def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        time.sleep(0.5)
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        if datetime.datetime.now() > end:
            break
            
doScrollDown(60)

In [51]:
title_list = []
url_list = []

for i in range(1,101):
    title = driver.find_element(By.XPATH,'//*[@id="_view_review_body_html"]/div/more-contents/div/ul/li[{}]/div[1]/div/a'.format(i)).text
    title2 = driver.find_element(By.XPATH,'//*[@id="_view_review_body_html"]/div/more-contents/div/ul/li[{}]/div[1]/div/a'.format(i))
    url = title2.get_attribute('href')
    
    title_list.append(title)
    url_list.append(url)
    if(i % 10 == 0):
        print(i)

10
20
30
40
50
60
70
80
90
100


In [52]:
new_doc = []

for i in range(0, len(url_list)):
    url_path = url_list[i]
    driver.switch_to.window(driver.window_handles[0])
    driver.execute_script("window.open('{}')".format(url_path))
    driver.switch_to.window(driver.window_handles[1])
    
    time.sleep(2)
    
    iframes = driver.find_elements(By.TAG_NAME, 'iframe')
    
    if len(iframes) > 0:
        driver.switch_to.frame('cafe_main')
        html = driver.page_source
        soup = BS(html, 'html.parser')
        try:
            a = soup.find('div', class_='article_viewer').get_text()
        except:
            a = 'null'
        new_doc.append(a)
        driver.switch_to.default_content()
    else:
        a = 'null'
        
    driver.close()
    time.sleep(1)
    if i%10 == 0:
        print(i)

0
10
20
30
40
50
60
70
80
90


In [53]:
raw_data = pd.DataFrame()
raw_data['title'] = title_list
raw_data['doc'] = new_doc
raw_data['url'] = url_list
raw_data['ch'] = 'naver'
raw_data['ch2'] = 'cafe'

In [54]:
f = open(file_path + "naver_cafe_.pkl", "wb")
pickle.dump(raw_data,f)

f.close()

In [55]:
driver.quit()

In [56]:
f = open(file_path + "naver_cafe_.pkl", "rb")
temp_file = pickle.load(f)
f.close()

temp_file

,title,doc,url,ch,ch2
0,[질문]국민연금 쪽에 아시는분...1억5천이 생겨요..,\n\n\n\n\n\n\n\n\n지금 말고....2036년도에 1억5천이 생기는 남...,https://cafe.naver.com/dohak27/1211685?art=ZXh...,naver,cafe
1,연금보험을 증권사로 돌리는 것?,\n\n\n\n\n\n\n\n\n달팽이 카페에서 여러 고수님들의 팁과 고견을 탐독중...,https://cafe.naver.com/pointns/365736?art=ZXh0...,naver,cafe
2,"기초생활생계급여,기초연금 궁금합니다.",[복지] 질문만 가능한 게시판입니다. 기본적인 정보는 적어주셔야 좀 더 정확한 답...,https://cafe.naver.com/cafe1535/784052?art=ZXh...,naver,cafe
3,해외이주로 인한 국민연금 환급 후기,\n\n\n\n\n\n\n\n\n안녕하세요. 2019년 3월 미국행 비행기를 타면서...,https://cafe.naver.com/gototheusa/787034?art=Z...,naver,cafe
4,퇴직했는데 국민연금을 계속 넣어야 할지 고민입니다ㅜㅜ,◈질문은 반드시 질문게시판에 작성 바랍니다. 댓글 작성자에게 EXP가 부여됩니다. ...,https://cafe.naver.com/hotellife/1689688?art=Z...,naver,cafe
...,...,...,...,...,...
95,자녀 연금보험 문의,✅ 월급쟁이 재테크 연구카페 보험전문가는 보험업법을 준수하며 정확한 정보와 정직한 ...,https://cafe.naver.com/onepieceholicplus/36185...,naver,cafe
96,개인 연금 잘 아시는분들 조언부탁드려요,※파우더룸에는 꼼꼼한 카페회원을 위한 공식 보험전문가가 배정되어 있습니다. ※꼭...,https://cafe.naver.com/cosmania/33970126?art=Z...,naver,cafe
97,"IRP, 개인퇴직연금저축 해지 질문드려요","\n\n\n\n\n\n\n\n\n​안녕하세요!인터넷 검색도 해보고, 고객센터문의도 ...",https://cafe.naver.com/onepieceholicplus/36136...,naver,cafe
98,"퇴직연금,,,,(기업 -> 개인IRP 입금)",\n\n\n\n\n\n\n\n\n< 아래 내용이 궁금합니다 회원님들의 지식을 나누어...,https://cafe.naver.com/serplove/743302?art=ZXh...,naver,cafe
